In [1]:
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

In [3]:
llm = ChatGroq(model="gemma2-9b-it", temperature=0)

In [4]:
def reverse_string(s):
    """Reverses the input string."""
    return s[::-1]

def capitalize_string(s):
    """Capsitizes the input string."""
    return s[:1].upper() + s[1:]

def to_lowercase(s):
    """Converts the input string to lowercase."""
    return s.lower()

def to_uppercase(s):
    """Converts the input string to uppercase."""
    return s.upper()


In [5]:
tools = [reverse_string, capitalize_string, to_lowercase, to_uppercase]
llm_with_tools = llm.bind_tools(tools)

In [10]:
sys_msg = SystemMessage(content="Your are a helpful assistant tasked with performing string functions")


In [6]:
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

#### Build Agent

In [7]:
# Graph
builder = StateGraph(MessagesState)
memory = MemorySaver()

# Define nodees: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edgesL these determine how the flow moves

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)

builder.add_edge("tools", "assistant")

react_graph = builder.compile(checkpointer=memory)


In [8]:
config = {"configurable": {"thread_id": "1"}}

In [11]:
messages = [HumanMessage(content="Reverse this string 'Charles McCullough'")]
messages = react_graph.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()


================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (call_mqxz)
 Call ID: call_mqxz
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

hguolluCcM selrahC


In [12]:
messages = [HumanMessage(content="Now convert the reversed string to upper case.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (call_mqxz)
 Call ID: call_mqxz
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (call_pwn3)
 Call ID: call_pwn3
  Args:
    s: hguolluCcM selrahC
================================= Tool Message =====

In [13]:
messages = [HumanMessage(content="Now reverse the string again and convert it to lowercase.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>{"tool_call":{"id":"call_y78j","type":"function","function":{"name":"reverse_string"},"parameters":{"s":"HGUOLLUCCM SELRAHC"}},"parameters":{"s":"HGUOLLUCCM SELRAHC"}}}</tool-use>\n'}}

In [14]:
messages = [HumanMessage(content="Now capitize the string.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (call_mqxz)
 Call ID: call_mqxz
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (call_pwn3)
 Call ID: call_pwn3
  Args:
    s: hguolluCcM selrahC
================================= Tool Message =====

In [15]:
print(capitalize_string("charles mccullough"))

Charles mccullough
